# Basic CNN

## CNN architecture

![cnn architecture](img/10-cnn-archi.png)


1. Do convolution with given input Image

2. As a result, many feature maps are produced

3. Subsampling which reduce information from convolution

4. We have series of linear convolution - subsampling.

5. All the information are connected to linear layer (= **FULLY CONNECTED or DENSE NET**)

    `typically, use Softmax classifier`
    

## Convolution 

`Concept is We are gonna look at only small portion of image(=**patch**), not all the pixels`

SO, look at one patch at a time


![simple-convolution](img/10-simple-convolution.png)


`Dot Product when calculating convolution`

simliar to Sliding window, we move patch by stride. 

and do convolution again, and we get **Feature Map**


![convnet](img/10-convnet-depth.png)


The number of filters we use, output feature map's depth.

in this case, we use 6 filters with size 5x5x3

So, we have 28x28 (because 32-5+1), and depth 6 sized feature map

Filer depth should be the same.


<br>

## Max Pooling


![max-pooling](img/10-max-pooling.png)

`idea : We want to **REUSE** the amount of information generated by convolution layers`

We chose each patch's maximum value and create feature maps


<br>
<hr>

## Simple CNN example

![simple CNN](img/10-simpleCNN.png)



In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


...


Test set: Average loss: 0.1724, Accuracy: 9497/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.135536
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.096274
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.203276
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.109775
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.158336
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.124747
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.059202
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.096011
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.144297
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.071255
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.149294
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.160912
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.328312
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.060570
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.078561
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.076950
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.167754
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.120635
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.170322
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.080251
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.060590
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.170389
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.093958
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.059514
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.079815
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.154423
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.095325
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.078436
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.115859
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.105210
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.042659
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.193458
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.113676
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.056633
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.044924
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.146790
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.165832
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.039059
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.128477


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.030264
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.092793
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.063274
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.108045
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.125679
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.069568
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.013783
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.038856
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.021135
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.081407
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.198052
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.132220
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.020862
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.148735
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.102836
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.046855
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.054007
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.008127
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.157738
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.088862


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.138721
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.093055
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.079869
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.043894
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.064105
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.107940
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.062813
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.287820
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.037110
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.045245
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.185269
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.057200
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.031295
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.025929
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.029754
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.056333
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.076577
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.095020
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.068387
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.009118


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.024420
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.058513
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.045022
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.047056
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.080558

Test set: Average loss: 0.0529, Accuracy: 9834/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.062065
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.045753
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.082742
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.044114
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.064968
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.288855
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.005068
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.057824
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.015106
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.090398
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.032476
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.079055
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.089037
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.006763
Train Epoch: 9 [